In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.utils.class_weight import compute_class_weight



In [14]:
merged_data = pd.DataFrame()
path = r'C:\Users\hp\Desktop\M2\PFE\Code\code pfe\dataset'
files = ['6.xlsx',
         '8.xlsx',
         '15.xlsx',
         '17.xlsx',
         '19.xlsx',
         '26.xlsx',
         '29.xlsx']
cols=['index','TVA (m3)', 'SPPA (kPa)',
       'MFOP ((m3/s)/(m3/s))', 'GASA (mol/mol)','STATUS','id']
for i in files:
    df=pd.read_excel(path+'\\'+i)
    df['id']=i[1]
    x=max(df[df['STATUS']==1].index)
    merged_data = merged_data.append(df[cols][:x], ignore_index=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_3540\3096263500.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(df[cols][:x], ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_3540\3096263500.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(df[cols][:x], ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_3540\3096263500.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_data = merged_data.append(df[cols][:x], ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_3540\3096263500.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged

In [17]:
merged_data.to_excel('data_kick.xlsx',index=False)

In [4]:


df=pd.read_excel(r'C:\Users\hp\Desktop\M2\PFE\Code\code pfe\dataset\all_data.xlsx')
df.shape

(110737, 7)

In [7]:
print(max(df[df['STATUS']==1].index))

106256


C:\Users\hp\AppData\Local\Temp\ipykernel_12652\1002051947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['index'] = pd.to_datetime(df['index'],format='%d/%m/%Y, %H:%M:%S')


In [43]:
data=data.rename(columns={'index': 'time'})

In [17]:
cols=['TVA (m3)', 'SPPA (kPa)',
       'MFOP ((m3/s)/(m3/s))', 'GASA (mol/mol)']

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17280 entries, 0 to 17279
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   time                  17280 non-null  datetime64[ns]
 1   TVA (m3)              17280 non-null  float64       
 2   SPPA (kPa)            17280 non-null  float64       
 3   MFOP ((m3/s)/(m3/s))  17280 non-null  float64       
 4   GASA (mol/mol)        17280 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 675.1 KB


In [50]:
import pandas as pd
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters


# Add a constant 'id' column for the single well
data['id'] = 1

# Convert your data to long format
long_data = data.melt(id_vars=["id", "time"], var_name="attribute", value_name="value")

long_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69120 entries, 0 to 69119
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         69120 non-null  int64         
 1   time       69120 non-null  datetime64[ns]
 2   attribute  69120 non-null  object        
 3   value      69120 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 2.1+ MB


In [49]:
# Extract features using tsfresh
extraction_settings = ComprehensiveFCParameters()
extracted_features = extract_features(long_data, column_id="id", column_sort="time", column_kind="attribute", column_value="value", default_fc_parameters=extraction_settings)

# Impute missing values in the extracted features
imputed_features = impute(extracted_features)

# Merge the extracted features with the original dataset
final_data = data.merge(imputed_features, on="id")

Feature Extraction:   0%|          | 0/4 [06:54<?, ?it/s]


MemoryError: Unable to allocate 4.45 GiB for an array with shape (17279, 17279, 2) and data type float64